# Week 01 - Lesson 05 - Building AI Agents with Google ADK

## Overview

This notebook introduces Google's Agent Development Kit (ADK), a powerful framework for building AI agents. You'll learn how ADK differs from other frameworks and how to create sophisticated single agents that can handle complex business scenarios.

# <img src="https://google.github.io/adk-docs/assets/adk-social-card.png" alt="Agent Development Kit" width="800"/>


### Learning Objectives
1. Understand ADK's framework
2. Create single agents with clear identity and purpose
3. Implement tools and instructions for agent behavior
4. Manage agent state and sessions

## Why Google ADK?

Google ADK is designed specifically for **enterprise AI agent development** with several key advantages:
- **Production-Ready**: Built for enterprise deployment and scaling
- **Google Cloud Integration**: Native integration with Google's AI services
- **Structured Approach**: Clear patterns for agent identity, tools, and behavior
- **Session Management**: Built-in state management and conversation tracking
- **Enterprise Security**: Designed with enterprise security and compliance in mind

## Key Concepts about the framework

1. **LlmAgent**: The core agent class for LLM-powered reasoning
2. **Tools**: Functions that agents can call to interact with external systems
3. **Instructions**: Clear guidance that shapes agent behavior and decision-making
4. **Sessions**: Persistent state management for ongoing conversations
5. **State**: Data that flows through your agent system

## 1. Environment Setup and Dependencies

First, let's ensure we have all necessary dependencies installed and configured.

In [ ]:
# Install required dependencies
%pip install -U --quiet google-adk google-genai litellm

## Getting Started with Google ADK

Google ADK allows you to use different LLM providers, such as Google Gemini, OpenAI, or Anthropic.  
You can use your **API key** from `Google AI Studio` (for Gemini), but you may also use your existing `OpenAI` keys if you have them.  
This enables your agents to call different LLMs as needed.

Before running the examples in this notebook, make sure you have configured **AT LEAST ONE** valid API key.

- 🔑 **OPENAI API KEY**: [Get you API Key from OpenAI Platform](https://platform.openai.com/settings/organization/api-keys)
Note: You will need to have credits in your OpenAI account to use the API key for LLM calls.


- 🔑 **GOOGLE AI STUDIO API Key**: [Get your FREE API Key from Google AI Studio](https://aistudio.google.com/apikey)  
You will need to sign in with your Google account and generate an API key.  
This key allows your agents to connect with **Gemini models** and other Google AI services.

👉 Configure your API keys in the environment before running the examples.

**Note:** To run the examples in this notebook, you only need to have at least one valid API key—either from OpenAI or from Google AI Studio (the latter is free).
You do not need both. As long as you have one of these API keys configured, you will be able to use the LLM features demonstrated here.


In [ ]:
import os

# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY" # <--- REPLACE

# OpenAI API Key (Get from OpenAI Platform: https://platform.openai.com/api-keys)
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY" # <--- REPLACE

# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No'}")

In [3]:
# Import necessary libraries
from google.adk.tools import FunctionTool
from typing import Dict, Any, List

from google.genai import types
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm


## 2. Understanding ADK Architecture

Before diving into code, let's understand how ADK differs from other frameworks:

### Core Differences from LangGraph

1. **Structured Identity**: Clear definition of agent purpose, capabilities, and constraints
2. **Tool Integration**: Seamless integration of external functions and APIs
3. **Session Management**: Built-in conversation state and context management
4. **Cloud Integration**: Native Google Cloud services integration

### ADK Agent Components

- **Identity**: Name, description, and purpose
- **Instructions**: Clear behavioral guidelines and constraints
- **Tools**: Functions the agent can call
- **Model**: The underlying LLM that powers reasoning
- **State Management**: Session-based conversation tracking

## 3. Setting Up the Foundation

Let's start by setting up the core components: the language model and our custom tools.

In [4]:
# Configuration constants
if os.environ.get('OPENAI_API_KEY') and os.environ.get('OPENAI_API_KEY') != "YOUR_OPENAI_API_KEY":
    # Import LiteLlm
    MODEL_GPT_4O = "openai/gpt-4.1" # You can also try: gpt-4.1-mini, gpt-4o etc.
    MODEL_NAME = LiteLlm(model=MODEL_GPT_4O)

elif os.environ.get('GOOGLE_API_KEY') and os.environ.get('OPENAI_API_KEY') != "YOUR_GOOGLE_API_KEY":
    MODEL_NAME = "gemini-2.0-flash"

else:
    raise ValueError("No API key found. Please set either OPENAI_API_KEY or GOOGLE_API_KEY in the environment.")

print("Model Name: ", MODEL_NAME)
APP_NAME = "adk_agent_app"
USER_ID = "user_001"
SESSION_ID = "session_001"

print("✅ Configuration set up successfully!")

Model Name:  model='openai/gpt-4.1' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x15d4fd5b0>
✅ Configuration set up successfully!


## 4. Creating Business Tools

Let's create some tools that our agent can use to solve business problems.

In [5]:
# Business Intelligence Tool
# Analyzes market data and provides insights
def analyze_market_trends(industry: str, region: str) -> Dict[str, Any]:
    """Analyze market trends for a specific industry and region."""
    
    # Simulated market analysis (in production, this would call real APIs)
    market_data = {
        "technology": {
            "north_america": {"growth_rate": "12.5%", "key_trends": ["AI/ML adoption", "Cloud migration", "Cybersecurity focus"]},
            "europe": {"growth_rate": "8.3%", "key_trends": ["GDPR compliance", "Digital transformation", "Green tech"]},
            "asia_pacific": {"growth_rate": "15.2%", "key_trends": ["Mobile-first", "E-commerce growth", "Fintech innovation"]}
        },
        "healthcare": {
            "north_america": {"growth_rate": "6.8%", "key_trends": ["Telemedicine", "AI diagnostics", "Personalized medicine"]},
            "europe": {"growth_rate": "5.2%", "key_trends": ["Universal healthcare", "Medical tourism", "Pharmaceutical innovation"]},
            "asia_pacific": {"growth_rate": "9.1%", "key_trends": ["Medical devices", "Healthcare IT", "Traditional medicine integration"]}
        }
    }
    
    if industry.lower() in market_data and region.lower().replace(" ", "_") in market_data[industry.lower()]:
        data = market_data[industry.lower()][region.lower().replace(" ", "_")]
        return {
            "industry": industry,
            "region": region,
            "growth_rate": data["growth_rate"],
            "key_trends": data["key_trends"],
            "analysis_timestamp": "2024-01-15T10:00:00Z"
        }
    else:
        return {
            "error": f"No data available for {industry} in {region}",
            "available_industries": list(market_data.keys()),
            "available_regions": ["North America", "Europe", "Asia Pacific"]
        }

print("✅ Market analysis tool created successfully!")

✅ Market analysis tool created successfully!


In [6]:
# Financial Analysis Tool
# Provides financial insights and recommendations
def analyze_financial_metrics(company_type: str, metrics: List[str]) -> Dict[str, Any]:
    """Analyze financial metrics for different company types."""
    
    # Simulated financial analysis
    financial_benchmarks = {
        "startup": {
            "burn_rate": "$50K-$200K/month",
            "runway": "12-18 months",
            "key_metrics": ["Monthly Recurring Revenue (MRR)", "Customer Acquisition Cost (CAC)", "Lifetime Value (LTV)"],
            "recommendations": ["Focus on product-market fit", "Optimize CAC", "Extend runway through funding"]
        },
        "scaleup": {
            "burn_rate": "$200K-$1M/month",
            "runway": "6-12 months",
            "key_metrics": ["Annual Recurring Revenue (ARR)", "Net Revenue Retention", "Gross Margin"],
            "recommendations": ["Scale efficiently", "Improve unit economics", "Focus on retention"]
        },
        "enterprise": {
            "burn_rate": "$1M+/month",
            "runway": "24+ months",
            "key_metrics": ["EBITDA", "Free Cash Flow", "Return on Invested Capital (ROIC)"],
            "recommendations": ["Optimize operations", "Strategic acquisitions", "Shareholder value creation"]
        }
    }
    
    if company_type.lower() in financial_benchmarks:
        data = financial_benchmarks[company_type.lower()]
        return {
            "company_type": company_type,
            "financial_profile": data,
            "requested_metrics": metrics,
            "analysis_timestamp": "2024-01-15T10:00:00Z"
        }
    else:
        return {
            "error": f"No data available for {company_type}",
            "available_types": list(financial_benchmarks.keys())
        }

print("✅ Financial analysis tool created successfully!")

✅ Financial analysis tool created successfully!


In [7]:
# Customer Insights Tool
# Analyzes customer behavior and provides insights
def analyze_customer_behavior(customer_segment: str, behavior_type: str) -> Dict[str, Any]:
    """Analyze customer behavior patterns for different segments."""
    
    # Simulated customer behavior analysis
    customer_insights = {
        "enterprise": {
            "purchase_behavior": {
                "decision_cycle": "3-6 months",
                "key_factors": ["ROI", "Integration complexity", "Vendor reputation"],
                "touchpoints": ["Sales team", "Product demos", "Case studies", "References"]
            },
            "support_preferences": {
                "channels": ["Dedicated account manager", "Phone support", "Email"],
                "response_time": "4 hours",
                "escalation_path": "Account manager → Sales director → VP"
            }
        },
        "mid_market": {
            "purchase_behavior": {
                "decision_cycle": "1-3 months",
                "key_factors": ["Cost-effectiveness", "Ease of use", "Quick implementation"],
                "touchpoints": ["Website", "Free trials", "Webinars", "Documentation"]
            },
            "support_preferences": {
                "channels": ["Email", "Chat support", "Knowledge base"],
                "response_time": "8 hours",
                "escalation_path": "Support team → Team lead → Manager"
            }
        },
        "startup": {
            "purchase_behavior": {
                "decision_cycle": "1-4 weeks",
                "key_factors": ["Speed to market", "Cost", "Flexibility"],
                "touchpoints": ["Social media", "Product hunt", "Referrals", "Free tier"]
            },
            "support_preferences": {
                "channels": ["Chat", "Discord/Slack", "Email"],
                "response_time": "24 hours",
                "escalation_path": "Community → Support → Founder"
            }
        }
    }
    
    if customer_segment.lower() in customer_insights:
        segment_data = customer_insights[customer_segment.lower()]
        if behavior_type.lower() in segment_data:
            return {
                "customer_segment": customer_segment,
                "behavior_type": behavior_type,
                "insights": segment_data[behavior_type.lower()],
                "analysis_timestamp": "2024-01-15T10:00:00Z"
            }
        else:
            return {
                "error": f"No {behavior_type} data available for {customer_segment}",
                "available_behaviors": list(segment_data.keys())
            }
    else:
        return {
            "error": f"No data available for {customer_segment}",
            "available_segments": list(customer_insights.keys())
        }

print("✅ Customer behavior analysis tool created successfully!")

✅ Customer behavior analysis tool created successfully!


## 5. Building Your First ADK Agent

Now let's create our first ADK agent. We'll build a Business Intelligence Agent that can analyze markets, finances, and customer behavior.

In [8]:
# Create FunctionTool instances from our functions
market_tool = FunctionTool(func=analyze_market_trends)
financial_tool = FunctionTool(func=analyze_financial_metrics)
customer_tool = FunctionTool(func=analyze_customer_behavior)

print("✅ Tools wrapped successfully!")
print(f"Market Analysis Tool: {market_tool.name}")
print(f"Financial Analysis Tool: {financial_tool.name}")
print(f"Customer Behavior Tool: {customer_tool.name}")

✅ Tools wrapped successfully!
Market Analysis Tool: analyze_market_trends
Financial Analysis Tool: analyze_financial_metrics
Customer Behavior Tool: analyze_customer_behavior


In [9]:
# Create the Business Intelligence Agent
business_intelligence_agent = LlmAgent(
    model=MODEL_NAME,
    name="business_intelligence_agent",
    description="A sophisticated business intelligence agent that analyzes markets, finances, and customer behavior to provide actionable insights for enterprise decision-making.",
    instruction="""
You are a Business Intelligence Agent specializing analysis of market, financial and customer behavior.

Your capabilities include:
1. **Market Analysis**: Analyze industry trends, growth rates, and key market drivers
2. **Financial Assessment**: Evaluate financial metrics and provide strategic recommendations
3. **Customer Insights**: Understand customer behavior patterns and preferences

When analyzing business scenarios:
- Always use the appropriate tools to gather data
- Provide structured, actionable insights
- Offer strategic recommendations based on data
- Maintain a professional, analytical tone

Use the available tools to:
- `analyze_market_trends`: For industry and regional market analysis
- `analyze_financial_metrics`: For company financial assessment
- `analyze_customer_behavior`: For customer segment insights

Always structure your responses with clear sections: Analysis, Key Insights, Strategic Implications, and Recommendations.
""",
    tools=[market_tool, financial_tool, customer_tool]
)

print("✅ Business Intelligence Agent created successfully!")
print(f"Agent Name: {business_intelligence_agent.name}")
print(f"Agent Description: {business_intelligence_agent.description}")
print(f"Number of Tools: {len(business_intelligence_agent.tools)}")

✅ Business Intelligence Agent created successfully!
Agent Name: business_intelligence_agent
Agent Description: A sophisticated business intelligence agent that analyzes markets, finances, and customer behavior to provide actionable insights for enterprise decision-making.
Number of Tools: 3


## 6. Setting Up Session Management

ADK provides robust session management to maintain conversation context and state. Let's set this up.

In [10]:
# Initialize session service and create a session
session_service = InMemorySessionService()

# Create a new session for our agent
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("✅ Session created successfully!")

✅ Session created successfully!


In [11]:
# Create a runner to execute our agent
runner = Runner(
    agent=business_intelligence_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("✅ Runner created successfully!")
print(f"Agent: {runner.agent.name}")
print(f"App: {runner.app_name}")
print(f"Session Service: {type(runner.session_service).__name__}")

✅ Runner created successfully!
Agent: business_intelligence_agent
App: adk_agent_app
Session Service: InMemorySessionService


## 7. Testing Your First Agent Interaction

Now let's test our Business Intelligence Agent with a real business scenario.

In [12]:
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    
    for event in events:
        if event.is_final_response() and event.content:
            print("\nFINAL RESPONSE")
            print("-" * 40)
            
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"📝 Text: {part.text}")
                elif hasattr(part, 'function_call'):
                    print(f"🔧 Function Call: {part.function_call}")
                else:
                    print(f"❓ Unknown Part Type: {type(part)}")
            
            return event.content.parts[0].text.strip() if event.content.parts else None
    return None

# Call the agent
call_agent("Analyze about helthcare market in Europe")


FINAL RESPONSE
----------------------------------------
📝 Text: Analysis:
The healthcare market in Europe is characterized by a robust annual growth rate of 5.2%. This region maintains expansive universal healthcare systems, significant pharmaceutical innovation, and is a major destination for medical tourism.

Key Insights:
- Universal healthcare coverage in most European countries drives stable demand for health services and products.
- Europe is a global leader in pharmaceutical research and development, fostering high rates of innovation.
- Medical tourism is growing, supported by high care standards and competitive pricing in specific countries.

Strategic Implications:
- Opportunities exist for firms specializing in pharmaceuticals, medtech, and digital health, particularly those that can navigate universal healthcare procurement systems.
- Partnering with regional healthcare providers and leveraging innovation in research and production can provide competitive advantages.
- Inv

"Analysis:\nThe healthcare market in Europe is characterized by a robust annual growth rate of 5.2%. This region maintains expansive universal healthcare systems, significant pharmaceutical innovation, and is a major destination for medical tourism.\n\nKey Insights:\n- Universal healthcare coverage in most European countries drives stable demand for health services and products.\n- Europe is a global leader in pharmaceutical research and development, fostering high rates of innovation.\n- Medical tourism is growing, supported by high care standards and competitive pricing in specific countries.\n\nStrategic Implications:\n- Opportunities exist for firms specializing in pharmaceuticals, medtech, and digital health, particularly those that can navigate universal healthcare procurement systems.\n- Partnering with regional healthcare providers and leveraging innovation in research and production can provide competitive advantages.\n- Investment in services or infrastructure supporting medi

## 8. ADK vs Other Frameworks: Key Advantages

Google’s **Agent Development Kit (ADK)** provides a structured, production-ready framework for building scalable and secure AI agents. 

> ADK is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.

### Key Advantages

- **Flexible Orchestration**  
  Combine deterministic workflow agents (Sequential, Parallel, Loop) with LLM-driven adaptive routing for both predictable pipelines and dynamic decision-making.

- **Hierarchical Multi-Agent Architecture**  
  Build modular applications with specialized agents arranged in clear hierarchies, enabling delegation, scalability, and complex workflows.

- **Rich Tool Ecosystem**  
  Use pre-built tools (Search, Code Exec), integrate custom functions or third-party libraries, and even chain other agents as callable tools.

- **Deployment Ready**  
  Scale seamlessly with **Vertex AI Agent Engine**, **Cloud Run**, or Dockerized infrastructure.

- **Built-in Evaluation**  
  Systematically test both final outputs and intermediate steps, ensuring reliability, compliance, and quality in enterprise settings.

- **Safety & Security by Design**  
  Incorporate guardrails, policies, and best practices to build trustworthy, enterprise-grade agents aligned with compliance standards.

---

✅ **When to Choose ADK**  
ADK is ideal for developers who need **hierarchical multi-agent systems**, seamless **Google Cloud integration**, and **built-in evaluation + deployment pipelines** — all within a framework that treats agents as reusable, software-like components.


## 9. Exercise: Build Your Own ADK Agent

Now it's your turn to create an ADK agent for a specific case scenario. Choose one of the following exercises:

In [13]:
# Exercise 1: Customer Success Agent
# Create an agent that helps customer success teams

def create_customer_success_agent():
    """Create a customer success agent."""
    
    # Create an agent that can:
    #   • Analyze customer health metrics
    #   • Provide onboarding recommendations
    #   • Suggest retention strategies
    #   • Handle common support queries
    
    # TODO: Implement your customer success agent here
    # 1. Define the agent's identity and purpose
    # 2. Create relevant tools for customer analysis
    # 3. Write comprehensive instructions
    # 4. Test with sample customer scenarios
    
    return "Customer success agent implementation goes here"

# Exercise 2: Financial Planning Agent
# Create an agent for financial planning and analysis

def create_financial_planning_agent():
    """Create a financial planning agent."""
        
    # Create an agent that can:
    #   • Analyze financial statements
    #   • Provide budgeting recommendations
    #   • Assess investment opportunities
    #   • Generate financial reports
    
    # TODO: Implement your financial planning agent here
    # 1. Define the agent's identity and purpose
    # 2. Create relevant tools for financial analysis
    # 3. Write comprehensive instructions
    # 4. Test with sample financial scenarios
    
    return "Financial planning agent implementation goes here"

## 10. Conclusion

Congratulations! You've successfully completed the lesson on Google ADK. You now understand:

### Technical Competencies
- How to create and configure ADK agents with clear identity and purpose
- How to implement tools and instructions for agent behavior
- How to manage agent sessions and state

### Next Steps

In future lessons, you'll learn about:

- Multi-agent systems with ADK
- Advanced tool integration and custom tools
- Deployment strategies for production

---

Happy coding with Google ADK! 🚀

- 📘 **Official Documentation**: [Google ADK Docs](https://google.github.io/adk-docs/)  
Explore the full reference, architecture guides, and advanced usage patterns.